In [5]:
import pandas as pd
import numpy as np
path = "beegfs/ECAI"

def get_intersection(v01, ve1, v02, ve2):
    # handle division by zero masking out the positions where the denominator is zero
    denominator = ve2 - ve1
    if isinstance(denominator, np.ndarray):
        denominator[denominator == 0] = np.nan
    else:
        if denominator == 0:
            return np.nan
    wes = (v01 - v02) / denominator
    return np.round(wes[(wes >= 0) & (wes <= 10)], 2)

In [6]:
# New method
folder = lambda db, eff, we : f"/db{db}_effrate{eff}_we{we}_ECAI"
unethical = pd.read_csv(path + folder(0, 0.2, 0)*2  + "/mo_rewards.csv")
unethical_np = unethical.to_numpy().reshape(-1, 5, 2)
ethical = pd.read_csv(path + folder(1, 0.2, 10)*2  + "/mo_rewards.csv")
ethical_np = ethical.to_numpy().reshape(-1, 5, 2)

# wes  = get_intersection(ethical_np[:, 0, 0], ethical_np[:, 0, 1], unethical_np[:, 0, 1], unethical_np[:, 0, 1])
wes = get_intersection(unethical_np[:, 0, 0], unethical_np[:, 0, 1], ethical_np[:, 0, 0], ethical_np[:, 0, 1])
wes.mean(), wes.std(), np.median(wes)

(2.4719976019184653, 1.6991766139698001, 2.18)

In [7]:
# Old method
folder = lambda db, eff, we : f"/db{db}_effrate{eff}_we{we}_ECAI"
unethical = pd.read_csv(path + folder(0, 0.2, 0)*2  + "/mo_rewards.csv")
unethical_np = unethical.to_numpy().reshape(-1, 5, 2).mean(axis=0)
ethical = pd.read_csv(path + folder(1, 0.2, 10)*2  + "/mo_rewards.csv")
ethical_np = ethical.to_numpy().reshape(-1, 5, 2).mean(axis=0)

wes  = get_intersection(unethical_np[0, 0], unethical_np[0, 1], ethical_np[0, 0], ethical_np[0, 1])
wes.mean(), wes.std()

(1.75, 0.0)

In [16]:
# By groups of 500

unethical = pd.read_csv(path + folder(0, 0.2, 0)*2  + "/mo_rewards.csv")
unethical_np = unethical.to_numpy().reshape(-1, 5, 2)
ethical = pd.read_csv(path + folder(1, 0.2, 10)*2  + "/mo_rewards.csv")
ethical_np = ethical.to_numpy().reshape(-1, 5, 2)
unethical_np = unethical_np.reshape(-1, 500, 5, 2)
ethical_np = ethical_np.reshape(-1, 500, 5, 2)

wes = []
for i in range(unethical_np.shape[0]):
    wes.append(get_intersection(unethical_np[i, :, 0, 0], unethical_np[i, :, 0, 1], ethical_np[i, :, 0, 0], ethical_np[i, :, 0, 1]))
    
for we in wes:
    print(we.mean().round(2), "+-", we.std().round(2), "Median" ,np.median(we))

2.37 +- 1.73 Median 2.08
2.53 +- 1.73 Median 2.23
2.48 +- 1.65 Median 2.2
2.49 +- 1.74 Median 2.09
2.42 +- 1.72 Median 2.09
2.34 +- 1.57 Median 2.12
2.52 +- 1.74 Median 2.15
2.59 +- 1.81 Median 2.34
2.47 +- 1.66 Median 2.2
2.5 +- 1.6 Median 2.31
